In [1]:
interp.load.ivy("org.typelevel" %% "cats" % "0.9.0")
interp.load.ivy("org.typelevel" %% "kittens" % "1.0.0-M9")
interp.load.ivy("com.chuusai" %% "shapeless" % "2.3.2")

In [2]:
import cats._
import cats.syntax._
import cats.implicits._
import cats.instances.all._
import cats.derived._
import cats.functor._
import scala.reflect._
import shapeless.HList._
// import shapeless.Functions._
import shapeless.Poly._
// import shapeless.TypeOperators._

import cats._

import cats.syntax._

import cats.implicits._

import cats.instances.all._

import cats.derived._

import cats.functor._

import scala.reflect._

import shapeless.HList._
// import shapeless.Functions._

import shapeless.Poly._
// import shapeless.TypeOperators._

# Typeclasses

In [ ]:
val shown = Show.apply[Map[String,Double]].show(Map("ajda" -> 3254.2345))
val shown1 = Map("ajda" -> 3254.2345).show

In [ ]:
import java.util.Date

implicit val dateShow = Show.show[Date] { date =>
    s"${date.getTime} milliseconds since epoch"
}

In [ ]:
new Date().show

In [ ]:
val eqInt = Eq[Int]
eqInt.eqv(123, 123)

123 =!= 123

Map("lol" -> List(1.2), "lel" -> List(2.1)) === Map("lel" -> List(2.1), "lol" -> List(1.2))

1.some === None

implicit val dateEq = Eq.instance[Date] { (x,y) => x.getTime == y.getTime }

new Date() === new Date() // weird mix of side effects, you see this now?!

# Monoids

In [ ]:
3.some |+| 4.some

3.some === none[Int]

implicit val dateMonoid = new Monoid[Date] {
    def empty = new Date(0)
    def combine(x:Date, y:Date):Date = new Date(x.getTime + y.getTime)
}

new Date() |+| new Date(36000000)

# Functors

In [ ]:
val f1 = (x:Int) => x.toDouble
val f2 = (y:Double) => y*3
val f3 = f1 map f2
f3(2)

import cats.data.NonEmptyList

val list = List(NonEmptyList(1, List(2, 3)), NonEmptyList(4, List(5, 6)))
val lifted = list.map(nel => Option(nel))
Monoid.combineAll(lifted)

val f = (x:Int) => Map("number" -> 1)
val flifted = Functor[Option].lift(f)
flifted(3.some)

In [ ]:
sealed trait Result[+A]
final case class Success[A](value:A) extends Result[A]
final case class Warning[A](value:A, message:String) extends Result[A]
final case class Failure[A](value:A) extends Result[Nothing]

def success[A](value:A):Result[A] = Success(value)
def warning[A](value:A, message:String):Result[A] = Warning(value, message)
def failure[A](value:A):Result[A] = Failure(value)

implicit val resultFunctor = new Functor[Result] {
    def map[A,B](x:Result[A])(f: A => B ):Result[B] =
        x match {
            case Success(x) => Success(f(x))
            case Warning(x, msg) => Warning(f(x), msg)
            case _:Result[_] => Failure()
        }
}

success(100) map( _ * 2 )

In [ ]:
val listOpt = List(1.some, 2.some, none)
Functor[List].compose(Functor[Option]).map(listOpt){ x => x + 3 }

import cats.data.Nested
val nested = Nested(listOpt)
nested.map{ _ + 3 }

# Applicatives

# Monads

In [ ]:
sealed trait Result[+A]
final case class Success[A](value:A) extends Result[A]
final case class Warning[A](value:A, message:String) extends Result[A]
final case class Failure[A](value:A) extends Result[Nothing]

def success[A](value:A):Result[A] = Success(value)
def warning[A](value:A, message:String):Result[A] = Warning(value, message)
def failure[A](value:A):Result[A] = Failure(value)

In [ ]:
implicit def resultMonad = new Monad[Result] {

    def flatMap[A, B](fa: Result[A])(f: A => Result[B]): Result[B] =
        fa match {
            case Success(x) => f(x)
            case Warning(x, m) => f(x)
            case Failure(x) => Failure(x)
        }

    def pure[A](x:A):Result[A] = Success(x)

    @annotation.tailrec
    def tailRecM[A, B](a: A)(f: A => Result[Either[A, B]]): Result[B] = f(a) match {
        case Failure(x) => Failure(x)
        case Success(Left(nextA)) => tailRecM(nextA)(f)
        case Success(Right(b)) => Success(b)

        case Warning(Left(nextA), msg) => tailRecM(nextA)(f)
        case Warning(Right(b), msg) => Warning(b, msg)
    }
}

warning(100, "whatever").flatMap{ x => Warning(x + 100, "whatever") }
Monad[Result].pure(121)

# Useful Monads

## Either

In [ ]:
val either:List[Either[Int, String]] = List(Right("1"), Left(2))

either.map{ e => e.flatMap{ x => Right(x.toString +" wow") }.map{ _ + " only to right" } }

Either.catchOnly[NumberFormatException]("abc".toInt)
Either.catchNonFatal("abc".toInt)

## Custom

In [ ]:
def coolEquationMonad[A[_] : Monad](a:Either[Int,String], b:Either[Int,String]): A[Either[Int,String]] = {
    val x = a.pure[A]
    val y = b.pure[A]
    x flatMap { x1 => y map { y1 => Right(x1.right.getOrElse(" ") + y1.right.getOrElse(" ")) } }
}

coolEquationMonad[Option](Right("bonny"), Left(1))

## ID

In [ ]:
val x:Id[Int] = 3
x.flatMap { _ + 2 }

In [ ]:
Right(1) flatMap { x => Right[String, Int](x) }

## Eval

In [8]:
val x = Eval.now{ 1 + 2 }
Eval.later{ 3 + 4 }
Eval.always{ 5 + 6 }
x.value

cmd8.sc:772: object Helper is not a member of package $sess
lazy val Helper: _root_.$sess.Helper.wrapper.type = _root_.$sess.Helper.wrapper
                              ^cmd8.sc:772: object Helper is not a member of package $sess
lazy val Helper: _root_.$sess.Helper.wrapper.type = _root_.$sess.Helper.wrapper
                                                                 ^cmd8.sc:836: object Helper is not a member of package $sess
lazy val Helper: _root_.$sess.Helper.wrapper.type = _root_.$sess.Helper.wrapper
                              ^cmd8.sc:836: object Helper is not a member of package $sess
lazy val Helper: _root_.$sess.Helper.wrapper.type = _root_.$sess.Helper.wrapper
                                                                 ^

: 

In [ ]:
val timedGreeting = Eval.always{ System.currentTimeMillis }.map{ x => s"Hello user, its ${x}" }
timedGreeting.value

In [ ]:
def stackDepth: Int = Thread.currentThread.getStackTrace.length
def loopM[M[_] : Monad](a:M[Int], b:Int): M[Int] = {
    println(s"Stack depth $stackDepth")
    b match {
        case 0 => a
        case n => a.flatMap{ _ => loopM(a, b-1) }
    }
}
loopM(1.some, 10)
println("-----------------------")
loopM(Eval.now(1), 10).value
println("-----------------------")
loopM(Eval.later(1), 10).value
println("-----------------------")
loopM(Eval.always(1), 10).value

## Writer

In [ ]:
import cats.data.Writer

val w = Writer(Vector("bhector"), 123)

type Logged[A] = Writer[Vector[String], A]
123.pure[Logged]
"whatever".pure[Logged]
val x = List("bhector").tell
x.run

In [ ]:
123.pure[Logged].map{ x => x + 100 }.flatMap{ x => (x - 100).pure[Logged] }.run

In [ ]:
val ret = 123.pure[Logged].bimap(
    log => log :+ "here", 
    result => result * 100
).bimap(
    log => log :+ " we", 
    result => result * 100
).bimap(
    log => log :+ " go", 
    result => result * 100
).mapWritten{ x => x.map{ _.toUpperCase } }

ret.run
ret.reset

## Reader

In [ ]:
import cats.data.Reader

val r1 = Reader( (x:Int) => x + 3 )
val r2 = Reader( (x:Int) => x + 2 )
r1(10)

val computation = r.map{ x => x*12 }.map{ x => x.toString }.run
computation(10)

val flatmapped = for {
    x <- r1
    y <- r2
} yield x + y
flatmapped.run.apply(10)

## Monad Transformers

In [ ]:
import cats.data.OptionT
import cats.data.EitherT

type ListOption[A] = OptionT[List, A]
42.pure[ListOption]  |+| 32.pure[ListOption]

In [ ]:
import scala.concurrent.Future
import scala.util.{ Success, Failure }
import scala.concurrent.ExecutionContext.Implicits.global

type Error = Exception
type ErrorOr[A] = Either[Error, A]
type FutureErrorOr[A] = EitherT[Future, Error, A]

val comp = (0 to 10).map{ x => x.pure[FutureErrorOr].map{ _ / 2 }.map{ x => x/x }.map{ _ + 10 } }

comp.map{ _.value.value match {
    case Some(Success(Right(x))) => x
    case _ => 0
} }

// 0.pure[ErrorOr].map{ _ + 1 }

In [ ]:
3.pure[FutureErrorOr]

In [ ]:
val a: String => String = _.toLowerCase
val b: String => String = _.toUpperCase
a.pure[FutureErrorOr].map{ _ andThen b }

In [ ]:
Either.catchNonFatal("0".toInt)

In [ ]:
trait WrappedFunction1[A, B] {
  def f : Function1[A, B]
  def doc: String
  def apply(x:A):B = f(x)

  def +[C](that:WrappedFunction1[B, C]):WrappedFunction1[A, C] = 
    new Wrapper[A, C](this.f andThen that.f, this.doc + " composed with " + that.doc)
}

class Wrapper[A, B](f1:Function1[A, B], sos:String) extends WrappedFunction1[A, B] {
  val f = f1
  val doc = sos
}

class ADifferentWrappedFunction1[A, B](f1:Function1[String, B], doc1:String) extends WrappedFunction1[String, B] {
  val (f, doc) = (f1, doc1)
}

object Wrapper {
  implicit class Wrap[A, B](f1:Function1[A, B]) {
    def wrap(sos:String):WrappedFunction1[A, B] = new Wrapper(f1, sos)
  }
    
  implicit class WrapS[A, B](f1:Function1[String, B]) {
      def wrapS(sos:String):ADifferentWrappedFunction1[String, B] = new ADifferentWrappedFunction1[String, B](f1, sos)
  }
}

In [ ]:
import Wrapper._
val x : String => String = _.toLowerCase
val y : String => String = _.toUpperCase
val z : String => List[String] = _.split(" ").toList
val x1 = x.wrap("a function for lowercasing")
val y1 = y.wrap("a function for uppercasing")
val z1 = z.wrap("a function for splitting a string")

In [4]:
import scala.language.higherKinds

import scala.language.higherKinds

In [5]:
case class Abstract[M[_], A, B](f:M[A] => M[B]) {
    def fn: M[A] => M[B] = { case x: M[A] => f(x) }
}

defined class Abstract

In [6]:
case class API[Mon[_] : Monad, I, O]() {
    type AM[T[_]] = Abstract[T, I, O]
}

defined class API

In [ ]:
def flatMap[M[A], M[B]](fa: Abstract[M[A]])(f: M[A] => Abstract[M[B]]): Abstract[M[B]]

In [3]:
import cats.free.Free

import cats.free.Free

In [7]:
case class FreeAPI[M[_]: Monad, I]() {
    type Abs[O] = Abstract[M, I, O]
}

defined class FreeAPI

In [8]:
val api = new FreeAPI[List, String]
import api._

api: FreeAPI[List, String] = FreeAPI()
import api._

In [8]:
val f = { case x:List[String] => x.map{ _.toLowerCase } }

cmd8.sc:772: object Helper is not a member of package $sess
lazy val Helper: _root_.$sess.Helper.wrapper.type = _root_.$sess.Helper.wrapper
                              ^cmd8.sc:772: object Helper is not a member of package $sess
lazy val Helper: _root_.$sess.Helper.wrapper.type = _root_.$sess.Helper.wrapper
                                                                 ^cmd8.sc:1: missing parameter type for expanded function
The argument types of an anonymous function must be fully known. (SLS 8.5)
Expected type was: ?
val f = { case x:List[String] => x.map{ _.toLowerCase } }
        ^cmd8.sc:824: object Helper is not a member of package $sess
lazy val Helper: _root_.$sess.Helper.wrapper.type = _root_.$sess.Helper.wrapper
                              ^cmd8.sc:824: object Helper is not a member of package $sess
lazy val Helper: _root_.$sess.Helper.wrapper.type = _root_.$sess.Helper.wrapper
                                                                 ^

: 